The main idea is to download pre-trained Imagenet model weights from the timm repository and fit Assymetric Laplace Distribution to the final fully connected layer weights. 

Please refer to my post https://sidml.github.io/An-interpretation-of-the-final-fully-connected-layer/ for more details about the project.

In [1]:
!pip install -q timm

In [2]:
!wget https://raw.githubusercontent.com/rwightman/pytorch-image-models/master/results/results-imagenet.csv

--2022-05-21 09:18:11--  https://raw.githubusercontent.com/rwightman/pytorch-image-models/master/results/results-imagenet.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40205 (39K) [text/plain]
Saving to: ‘results-imagenet.csv’

results-imagenet.cs 100%[===================>]  39.26K  --.-KB/s    in 0.004s  

2022-05-21 09:18:11 (9.08 MB/s) - ‘results-imagenet.csv’ saved [40205/40205]



In [3]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from collections import OrderedDict
from tqdm.auto import tqdm
from glob import glob
import pdb, json, timm, os

## Load Imagenet class names

In [4]:
def setup_imagenet_classes(data_dir = "../input/imagenetval"):
        
    # Read the categories
    with open(f"{data_dir}/imagenet_classes.txt", "r") as f:
        categories = [s.strip().lower() for s in f.readlines()]
    
    # https://gist.github.com/aaronpolhamus/964a4411c0906315deb9f4a3723aac57
    cls_map = pd.read_csv(f"{data_dir}/map_clsloc.txt", sep=' ', header=None)
    cls_map.columns = ['imagenet_label', 'imagenet_clsnum', 'string_label']

    with open(f"{data_dir}/imagenet_class_index.json") as data_file:    
        data = json.load(data_file)              
            
    cls_stats = pd.DataFrame(data.values(), index=data.keys(),
                      columns=['imagenet_label', 'string_label'])
    cls_stats.index.name = 'pytorch_clsnum'
    cls_stats = cls_stats.reset_index()
    cls_stats = cls_stats.set_index('imagenet_label')

    cls_stats.loc[cls_map.imagenet_label, "imagenet_clsnum"] = np.arange(1, 1001)
    cls_stats['imagenet_clsnum'] = cls_stats['imagenet_clsnum'].astype(int)
    
    val_sol = pd.read_csv(f"{data_dir}/LOC_train_solution.csv")
    # val_sol = pd.read_csv("LOC_val_solution.csv")
    all_rows = val_sol.PredictionString.apply(lambda x: x.strip().split(' '))

    class_counts = val_sol.ImageId.apply(lambda x: x.split('_')[0]).value_counts()
    class_counts.name = "cls_count"
    class_counts.index.name = 'imagenet_label'
    cls_stats = cls_stats.merge(class_counts, on='imagenet_label')

    cls_stats['pytorch_clsnum'] = cls_stats['pytorch_clsnum'].astype(int)


    if cls_stats.index.name!='pytorch_clsnum':
        cls_stats = cls_stats.reset_index()
        cls_stats = cls_stats.set_index('pytorch_clsnum')

    return cls_stats

## Extract Final fully connected layer weights

In [5]:
@torch.no_grad()
def setup_weights(model_name):
    model = timm.create_model(model_name, pretrained=True)    
    state = model.state_dict()        
    # last two layers should be fc
    layer_name = list(state.keys())[-2:]
    try:
        fc_w = state[layer_name[0]].cpu().numpy()
        if len(fc_w.shape)!=2:
            print(layer_name, fc_w.shape)
            print('unable to extract fc layer')
            return None, None
        if len(layer_name)==2:
            fc_b = state[layer_name[1]].cpu().numpy()
        else:
            fc_b = None
    except Exception as e:
        print('couldnt process', model_name)
        print(e)
        return None, None
    return fc_w, fc_b

## Fit Asymmetric Laplace Distribution

In [6]:
def fit_laplace(y, fig_name=None):
    y = np.sort(y)
    x = np.arange(len(y))/len(y)
    target = np.log(y)
    coeff = np.polyfit(x, target, 1)
    predict_y = np.poly1d(coeff)(x)
    r2 = r2_score(target, predict_y)
    if fig_name:
        cls_name = fig_name.split('/')[-1]
        plt.plot(predict_y); plt.plot(target)    
        plt.legend(['Predicted', 'log(Actual Weights)'])
        plt.title(f"Class :{cls_name}, R2: {r2:.3f}"); plt.tight_layout();
        plt.savefig(f"{fig_name}.jpg"); plt.clf()
    return coeff[0], coeff[1], r2

def fit_joint_laplace(fc_w):
    pos_weights = fc_w[fc_w > 0]
    neg_weights = -fc_w[fc_w < 0]

    slope_pos, int_pos, r2_pos = fit_laplace(pos_weights)
    slope_neg, int_neg, r2_neg = fit_laplace(neg_weights)
    
    # estimate dataset mean assuming joint distribution of iid asymmetric laplace distribution
    k_est_pos = np.sqrt(np.exp(int_pos)/(slope_pos-np.exp(int_pos)))
    laplace_lam_est_pos = slope_pos * k_est_pos
    mean_pos = (1-k_est_pos**2)/(laplace_lam_est_pos*k_est_pos)


    k_est_neg = np.sqrt(np.exp(int_neg)/(slope_neg-np.exp(int_neg)))
    laplace_lam_est_neg = slope_neg * k_est_neg
    mean_neg = (1-k_est_neg**2)/(laplace_lam_est_neg*k_est_neg)

    print(f"joint fit: mean_pos {mean_pos:.2f}, r2_pos {r2_pos:.2f}")
    print(f"joint fit: mean_neg {mean_neg:.2f}, r2_neg {r2_neg:.2f}")

    return
    

def fit_exp(cls_stats_df, name, fc_w):
    
    # reset previous values if they exist and write new values
    cls_stats_df.loc[:, ['slope_pos', 'int_pos', 'r2_pos',\
                      'slope_neg', 'int_neg', 'r2_neg']] = None
    
    os.makedirs(f"./{name}/pos", exist_ok=True)
    os.makedirs(f"./{name}/neg", exist_ok=True)

    plt.figure()
    for pytorch_clsnum in range(fc_w.shape[0]):
        y = fc_w[pytorch_clsnum]
        y_pos, y_neg = y[y>=0], -1*y[y<0]
        fig_name = f"./{name}/pos/{cls_stats_df.loc[pytorch_clsnum, 'string_label']}"
        cls_stats_df.loc[pytorch_clsnum, ['slope_pos', 'int_pos', 'r2_pos']] = fit_laplace(y_pos, fig_name)
#         fig_name = f"./{name}/neg/{cls_stats_df.loc[pytorch_clsnum, 'string_label']}"
        cls_stats_df.loc[pytorch_clsnum, ['slope_neg', 'int_neg', 'r2_neg']] = fit_laplace(y_neg, fig_name=None)
    plt.close()
    
                         
    all_coeff_pos = cls_stats_df.loc[:, ['slope_pos', 'int_pos']].astype(float).values
    all_coeff_neg = cls_stats_df.loc[:, ['slope_neg', 'int_neg']].astype(float).values
                         
    # estimate mean for each class which is asymmetric laplace distribution
    # take mean of all estimated laplace means to obtain overall mean of the dataset
    k_est_pos = np.sqrt(np.exp(all_coeff_pos[:, 1])/(all_coeff_pos[:, 0]-np.exp(all_coeff_pos[:, 1])))
    laplace_lam_est_pos = all_coeff_pos[:, 0] * k_est_pos
    mean_pos = (1-k_est_pos**2)/(laplace_lam_est_pos*k_est_pos)


    k_est_neg = np.sqrt(np.exp(all_coeff_neg[:, 1])/(all_coeff_neg[:, 0]-np.exp(all_coeff_neg[:, 1])))
    laplace_lam_est_neg = all_coeff_neg[:, 0] * k_est_neg
    mean_neg = (1-k_est_neg**2)/(laplace_lam_est_neg*k_est_neg)

    marginal_mean_pos, marginal_mean_neg = mean_pos.mean(), mean_neg.mean()
    print(f"marginal_mean_pos:{marginal_mean_pos:.2f}, marginal_mean_neg:{marginal_mean_neg:.2f}")

    # estimate joint laplace
    fit_joint_laplace(fc_w)
    
    plt.figure()
    coeff = np.polyfit(all_coeff_pos[:, 0], all_coeff_pos[:, 1], 1)
    plt.scatter(all_coeff_pos[:, 0], all_coeff_pos[:, 1])
    plt.title(f"slope:{coeff[0]:.2f}, intercept:{coeff[1]:.2f}")
    plt.savefig(f"./{name}/pos_coeff_scatter.jpg")
    plt.close()
                         
    return cls_stats_df

In [7]:
timm_results = pd.read_csv('results-imagenet.csv')
model_names = timm_results.model.values
weight_dir = "/root/.cache/torch/hub/checkpoints/"

## Kaggle kernels have a disk usage limit, so we can't process all the available models on timm. We will randomly select 75 models for our analysis. Please feel free to run & share the results for all the models if you have sufficient compute.
Most of the time is taken while generating the plots. We are saving ALD plots for all the 1000 imagenet classes for every model architecture.

In [8]:
cls_stats_df = setup_imagenet_classes()
os.makedirs('./exp_fit_data/', exist_ok=True)
model_names = np.random.choice(model_names, 75)
for model_name in model_names:
    print("\nprocessing:", model_name)
    try:
        fc_w, fc_b = setup_weights(model_name)
        if fc_w is None:
            print('unable to extract weights')
            continue
        cls_stats_df = fit_exp(cls_stats_df, model_name, fc_w)
        cls_stats_df.to_csv(f"./exp_fit_data/{model_name}.csv")
    except Exception as e:
        print(e)
        continue    
    # clear the weights otherwise kernel will run out of memory
    for f in glob(f"{weight_dir}/{model_name}*.pth"):
        os.remove(f)   


processing: gluon_resnet101_v1s


Downloading: "https://github.com/rwightman/pytorch-pretrained-gluonresnet/releases/download/v0.1/gluon_resnet101_v1s-60fe0cc1.pth" to /root/.cache/torch/hub/checkpoints/gluon_resnet101_v1s-60fe0cc1.pth
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


Input contains NaN, infinity or a value too large for dtype('float32').

processing: dm_nfnet_f3


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-dnf-weights/dm_nfnet_f3-d74ab3aa.pth" to /root/.cache/torch/hub/checkpoints/dm_nfnet_f3-d74ab3aa.pth


marginal_mean_pos:522.61, marginal_mean_neg:392.78
joint fit: mean_pos 519.62, r2_pos 0.87
joint fit: mean_neg 393.91, r2_neg 0.81

processing: gluon_resnext101_32x4d


...

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/mixnet_m-4647fc68.pth" to /root/.cache/torch/hub/checkpoints/mixnet_m-4647fc68.pth


marginal_mean_pos:217.96, marginal_mean_neg:179.54
joint fit: mean_pos 216.34, r2_pos 0.84
joint fit: mean_neg 180.98, r2_neg 0.81


<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

### Zip the generated results and clean directory

In [9]:
!zip -rq out.zip ./

In [10]:
import shutil
all_dir = glob("./*/")
for dir in all_dir:
    shutil.rmtree(dir)